# V2 Model Server (SKLearn)

In [1]:
import mlrun

### Configuration and package dependencies

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage="mlrun/ml-models"

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


### Serving class code

In [3]:
from cloudpickle import load
from typing import List
from sklearn.datasets import load_iris
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [4]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [5]:
# nuclio: end-code

# Convert to function object

### Document and save (as a template)

In [6]:
import mlrun
fn = mlrun.code_to_function('v2-model-server', 
                            description="generic sklearn model server",
                            categories=['serving', 'ml'],
                            labels={'author': 'yaronh', 'framework': 'sklearn'},
                            code_output='.')

fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2020-11-24 16:17:53,525 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [13]:
models_path = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'
mlrun.mlconf.dbpath = mlrun.mlconf.dbpath or 'http://mlrun-api:8080'
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True

# Test models locally (using a server emulator)

In [14]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server(globals())

{'model_path': 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'}
> 2020-11-24 16:19:02,128 [info] model mymodel was loaded
> 2020-11-24 16:19:02,129 [info] Loaded ['mymodel']


### Test against the iris dataset 

In [15]:
iris = load_iris()
x = iris['data'].tolist()

In [16]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [17]:
fn.apply(mlrun.mount_v3io())

In [18]:
fn.deploy()

> 2020-11-24 16:19:05,505 [warning] warning!, server (0.5.4-rc1) and client (0.5.4) ver dont match
> 2020-11-24 16:19:05,505 [info] Starting remote function deploy
2020-11-24 16:19:05  (info) Deploying function
2020-11-24 16:19:05  (info) Building
2020-11-24 16:19:05  (info) Staging files and preparing base images
2020-11-24 16:19:05  (info) Building processor image
2020-11-24 16:22:12  (info) Build complete
2020-11-24 16:22:18  (info) Function deploy complete
> 2020-11-24 16:22:18,854 [info] function deployed, address=default-tenant.app.dsteam.iguazio-cd1.com:30984


'http://default-tenant.app.dsteam.iguazio-cd1.com:30984'

##  Test server

In [19]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
fn.invoke('/v2/models/mymodel/infer', my_data)

{'id': 'a34730cf-c25a-4f30-b323-214d8d3033c7',
 'model_name': 'mymodel',
 'outputs': [0, 2]}